# Calculates the environmental temperature and lapse rate from COSMIC data

Author: Brian Green (briangre@stanford.edu)

In [1]:
import matplotlib.pyplot as plt
from matplotlib.ticker import ScalarFormatter
import numpy as np
import pandas as pd
from dataclasses import dataclass
import warnings
from scipy.ndimage import label
from scipy.stats import chi2
from scipy import interpolate
from tqdm.notebook import tqdm
with warnings.catch_warnings():
    tqdm.pandas()
warnings.simplefilter("ignore")
    
%matplotlib inline

import xarray as xr
import dask
import os

from time import time
from scipy.spatial import Delaunay
import datetime

DLAT_THRESH = 2.5 # (degrees latitude) the distance away from the balloon to look for COSMIC profiles
K_THRESH = 6 # the zonal wavenumber threshold that sets DLON_THRESH
DLON_THRESH = 360/K_THRESH # (degrees longitude) the distance away from the balloon to look for COSMIC profiles
DT_THRESH = pd.Timedelta(1, 'days') # the +/- time window to look for COSMIC profiles
TIME_REF = pd.Timestamp('2014') # the reference time for converting timestamps to floats
TIME_UNITS = pd.Timedelta(1, 'days') # the unit of time all the timestamps will be converted to
INTERP_METHOD = 'linear' # the interpolation method to estimate T and dTdz at the balloon altitude
T_COSMIC_DUMMY = -1E6 # (degC) the temperature value that's used for a flight if that flight's N_cosmic = 0
DTDZ_COSMIC_DUMMY = -1E6 # (K/m) the lapse rate value that's used for a flight if that flight's N_cosmic = 0

MIN_SEG_LEN = pd.Timedelta(6, 'hours')    # the minimum segment length

### Load in the raw flight data and the list of COSMIC profiles

In [2]:
temp1 = pd.read_feather('temp_data/2_segments_2020_Jan-Jun.feather')
temp2 = pd.read_feather('temp_data/2_segments_2020_Jul-Dec.feather')
temp3 = pd.read_feather('temp_data/2_segments_2021.feather')

# Increment the segment IDs from one file to another, so they are still unique
temp2.segment_id = temp2.segment_id + temp1.segment_id.max() + 1
temp3.segment_id = temp3.segment_id + temp2.segment_id.max() + 1

frames = [temp1, temp2, temp3]
flights = pd.concat(frames, ignore_index = True)

In [3]:
#flights = pd.read_feather('temp_data/2_segments_2020_Jan-Jun.feather')
cosmic_list = pd.read_feather('temp_data/3a_cosmic_list.feather')

# Initialize temperature, lapse rate, and number of COSMIC profiles variables
flights['T_cosmic'] = np.nan
flights['dTdz_cosmic'] = np.nan
flights['N_cosmic'] = 0

In [4]:
# Select just the segments in 2021
segment_time_start = flights.groupby('segment_id').progress_apply(lambda f: f.time.iloc[0])
segment_ids = segment_time_start[segment_time_start < pd.Timestamp('2022-01-01 00:00:00')]
segment_ids = segment_ids[segment_ids >= pd.Timestamp('2021-01-01 00:00:00')]
flights = flights[flights.segment_id.isin(segment_ids.index)]

# Select just the segments longer than MIN_SEG_LEN
print('Selecting segments longer than', MIN_SEG_LEN)
nsegments_before = flights.segment_id.nunique()
segment_length = flights.groupby('segment_id').progress_apply(lambda f: f.time.iloc[-1] - f.time.iloc[0])
segments_selected = segment_length[segment_length >= MIN_SEG_LEN]
flights = flights[flights.segment_id.isin(segments_selected.index)]
print('Selected', flights.segment_id.nunique(), 'out of', nsegments_before, 'segments')

  0%|          | 0/60545 [00:00<?, ?it/s]

Selecting segments longer than 0 days 06:00:00


  0%|          | 0/11386 [00:00<?, ?it/s]

Selected 1560 out of 11386 segments


### Do some processing of the times and longitudes

Times need to be converted to float numbers; "fake" longitudes need to be created to account for the date line.

In [5]:
# The new variable time_float is defined as the number of TIME_UNITS after TIME_REF
cosmic_list['time_float'] = (cosmic_list.Date - TIME_REF)/TIME_UNITS
flights['time_float'] = (flights.time - TIME_REF)/TIME_UNITS

# Make a second list of the COSMIC profiles that are within +/- DLON_THRESH of the date line
# Longitudes between 180 and 180-DLON_THRESH get 360 degrees added:
temp1 = cosmic_list[cosmic_list.Lon >= 180 - DLON_THRESH]
temp1.Lon = temp1.Lon - 360
# Longitudes between -180 and -180+DLON_THRESH get 360 degrees subtracted
temp2 = cosmic_list[cosmic_list.Lon <= -180 + DLON_THRESH]
temp2.Lon = temp2.Lon + 360
# Append
cosmic_list = cosmic_list.append(temp1, ignore_index=True)
cosmic_list = cosmic_list.append(temp2, ignore_index=True)

In [6]:
flights

,flight_id,time,latitude,longitude,altitude,pressure,velocity_u,velocity_v,suspicious_motion,segment_id,T_cosmic,dTdz_cosmic,N_cosmic,time_float
1222570,LN-127,2021-01-14 13:28:35,3.774244,-76.600244,18979.6,65.77,1.288,-1.023,0,5135,NaN,NaN,0,2570.561516
1222571,LN-127,2021-01-14 13:29:36,3.773697,-76.599301,18961.6,66.00,1.126,-0.942,0,5135,NaN,NaN,0,2570.562222
1222572,LN-127,2021-01-14 13:30:36,3.772939,-76.598550,18961.4,65.95,0.993,-0.841,0,5135,NaN,NaN,0,2570.562917
1222573,LN-127,2021-01-14 13:31:23,3.772493,-76.598242,18964.5,65.95,0.933,-0.820,0,5135,NaN,NaN,0,2570.563461
1222574,LN-127,2021-01-14 13:31:37,3.772393,-76.598174,18968.5,65.93,0.899,-0.800,0,5135,NaN,NaN,0,2570.563623
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16082875,LN-325,2021-03-06 11:37:20,11.197386,-135.131277,17415.5,84.49,-2.421,4.332,0,60851,NaN,NaN,0,2621.484259
16082876,LN-325,2021-03-06 11:38:20,11.199723,-135.132479,17411.7,84.52,-2.446,4.314,0,60851,NaN,NaN,0,2621.484954
16082877,LN-325,2021-03-06 11:38:33,11.200217,-135.132738,17413.3,84.51,-2.472,4.336,0,60851,NaN,NaN,0,2621.485104
16082878,LN-325,2021-03-06 11:39:20,11.202002,-135.133763,17420.1,84.42,-2.513,4.351,0,60851,NaN,NaN,0,2621.485648


In [7]:
cosmic_list

,Name,Lat,Lon,Date,time_float
0,/Users/briangre/Documents/gravity_waves/cosmic...,48.838894,-20.548496,2013-05-15 17:32:00,-230.269444
1,/Users/briangre/Documents/gravity_waves/cosmic...,-43.159657,-77.366478,2013-05-15 11:56:00,-230.502778
2,/Users/briangre/Documents/gravity_waves/cosmic...,23.364857,-28.325857,2013-05-15 14:35:00,-230.392361
3,/Users/briangre/Documents/gravity_waves/cosmic...,55.125278,45.992245,2013-05-15 02:41:00,-230.888194
4,/Users/briangre/Documents/gravity_waves/cosmic...,67.952621,-149.315048,2013-05-15 23:17:00,-230.029861
...,...,...,...,...,...
7501099,/Users/briangre/Documents/gravity_waves/cosmic...,22.303007,206.728760,2021-04-23 04:04:00,2669.169444
7501100,/Users/briangre/Documents/gravity_waves/cosmic...,22.058485,221.683365,2021-04-23 13:13:00,2669.550694
7501101,/Users/briangre/Documents/gravity_waves/cosmic...,35.165123,231.865387,2021-04-23 18:55:00,2669.788194
7501102,/Users/briangre/Documents/gravity_waves/cosmic...,-7.088166,224.700745,2021-04-23 17:18:00,2669.720833


### Functions that identify the COSMIC profiles to use and calculate the environmental temperature and lapse rate

In [8]:
def flight_find_cosmic(flight, cosmic_list):
    
    # Valid COSMIC profiles are those inside a 3D tunnel in lat, lon, time space
    # that the balloon passes (approximately) through the center of.
    
    # The tunnel is defined using a point cloud. The points in the cloud are the corners
    # of boxes that are calculated at each (lat, lon, time): (lat +/- DLAT_THRESH,
    # lon +/- DLON_THRESH, time +/- DT_THRESH).
    
    # The tunnel is constructed by using a Delaunay tesselation of the point cloud, and
    # the list COSMIC profiles is searched for profiles that lie inside the tunnel. Since 
    # the Delaunay tesselation method draws straight lines to define the boundaries of convex
    # shapes, it creates a tunnel that is too "wide" and will result in too many profiles
    # included in the list. This is ok, because a similar approach will be used to identify the
    # profiles at each time step using the boxes as defined above.
    
    # Sometimes the balloon crosses the date line, which makes the tunnel shape too complex.
    # If this happens, construct separate tunnels for each side of the dateline for each time
    # the balloon crosses it. (The longitude jump will be ~360 degrees, but I set the threshold to 180.)
    if flight.longitude.diff().abs().max() > 180:
        
        cosmic_to_load = pd.DataFrame()
        flight = flight.reset_index(drop = True) # so that the index lines up with the corner row numbers
        flight['dateline_id'] = (flight.longitude.diff().abs() > 180).astype(int).cumsum()
        dateline_ids = flight.dateline_id.unique()
        
        for i in np.arange(flight.dateline_id.nunique()):
            
            temp_flight = flight[flight.dateline_id == dateline_ids[i]].copy()
            
            
            temp = temp_flight[['latitude','longitude','time_float']].copy()
            corner1 = np.array([temp.latitude.min() - DLAT_THRESH,
                                temp.longitude.min() - DLON_THRESH,
                                temp.time_float.min() - DT_THRESH/TIME_UNITS])
            corner2 = np.array([temp.latitude.min() - DLAT_THRESH,
                                temp.longitude.max() + DLON_THRESH,
                                temp.time_float.min() - DT_THRESH/TIME_UNITS])
            corner3 = np.array([temp.latitude.max() + DLAT_THRESH,
                                temp.longitude.max() + DLON_THRESH,
                                temp.time_float.min() - DT_THRESH/TIME_UNITS])
            corner4 = np.array([temp.latitude.max() + DLAT_THRESH,
                                temp.longitude.min() - DLON_THRESH,
                                temp.time_float.min() - DT_THRESH/TIME_UNITS])
            corner5 = np.array([temp.latitude.min() - DLAT_THRESH,
                                temp.longitude.min() - DLON_THRESH,
                                temp.time_float.max() + DT_THRESH/TIME_UNITS])
            corner6 = np.array([temp.latitude.min() - DLAT_THRESH,
                                temp.longitude.max() + DLON_THRESH,
                                temp.time_float.max() + DT_THRESH/TIME_UNITS])
            corner7 = np.array([temp.latitude.max() + DLAT_THRESH,
                                temp.longitude.max() + DLON_THRESH,
                                temp.time_float.max() + DT_THRESH/TIME_UNITS])
            corner8 = np.array([temp.latitude.max() + DLAT_THRESH,
                                temp.longitude.min() - DLON_THRESH,
                                temp.time_float.max() + DT_THRESH/TIME_UNITS])
            
            box = np.stack((corner1,corner2,corner3,corner4,
                            corner5,corner6,corner7,corner8))
            
            # Do the Delaunay tesselation and see which COSMIC profiles lie within the tunnel.
            # Points outside the tesselation are labeled with a -1, so test whether the label
            # is greater than or equal to zero.
            temp = np.array(cosmic_list[['Lat','Lon','time_float']])
            inside = Delaunay(box).find_simplex(temp) >= 0
            
            
            # Make a list of the COSMIC profiles and append
            temp = cosmic_list.copy()
            temp['inside'] = inside
            temp = temp[temp.inside == True].drop(['inside'], axis=1)
            cosmic_to_load = cosmic_to_load.append(temp, ignore_index=True)
    
    else:
        
        temp = flight[['latitude','longitude','time_float']].copy()
        corner1 = np.array([temp.latitude.min() - DLAT_THRESH,
                            temp.longitude.min() - DLON_THRESH,
                            temp.time_float.min() - DT_THRESH/TIME_UNITS])
        corner2 = np.array([temp.latitude.min() - DLAT_THRESH,
                            temp.longitude.max() + DLON_THRESH,
                            temp.time_float.min() - DT_THRESH/TIME_UNITS])
        corner3 = np.array([temp.latitude.max() + DLAT_THRESH,
                            temp.longitude.max() + DLON_THRESH,
                            temp.time_float.min() - DT_THRESH/TIME_UNITS])
        corner4 = np.array([temp.latitude.max() + DLAT_THRESH,
                            temp.longitude.min() - DLON_THRESH,
                            temp.time_float.min() - DT_THRESH/TIME_UNITS])
        corner5 = np.array([temp.latitude.min() - DLAT_THRESH,
                            temp.longitude.min() - DLON_THRESH,
                            temp.time_float.max() + DT_THRESH/TIME_UNITS])
        corner6 = np.array([temp.latitude.min() - DLAT_THRESH,
                            temp.longitude.max() + DLON_THRESH,
                            temp.time_float.max() + DT_THRESH/TIME_UNITS])
        corner7 = np.array([temp.latitude.max() + DLAT_THRESH,
                            temp.longitude.max() + DLON_THRESH,
                            temp.time_float.max() + DT_THRESH/TIME_UNITS])
        corner8 = np.array([temp.latitude.max() + DLAT_THRESH,
                            temp.longitude.min() - DLON_THRESH,
                            temp.time_float.max() + DT_THRESH/TIME_UNITS])
            
        box = np.stack((corner1,corner2,corner3,corner4,
                        corner5,corner6,corner7,corner8))
            
        # Do the Delaunay tesselation and see which COSMIC profiles lie within the tunnel.
        # Points outside the tesselation are labeled with a -1, so test whether the label
        # is greater than or equal to zero.
        temp = np.array(cosmic_list[['Lat','Lon','time_float']])
        inside = Delaunay(box).find_simplex(temp) >= 0
        
        cosmic_to_load = cosmic_list.copy()
        cosmic_to_load['inside'] = inside
        cosmic_to_load = cosmic_to_load[cosmic_to_load.inside == True].drop(['inside'], axis=1)
    
    return cosmic_to_load
    

In [9]:
def calc_backgroundT(flight, cosmic_to_load, temperatures, altitude):
    
    # Make the box corners for each data point (all at once)
    # See the description in the fin_flight_cosmic function for 
    # what the "box" is and what it's for.
    temp = np.array(flight[['latitude','longitude','time_float']])
    corner1 = temp.copy()
    corner1[:,0] = corner1[:,0] - DLAT_THRESH
    corner1[:,1] = corner1[:,1] - DLON_THRESH
    corner1[:,2] = corner1[:,2] - DT_THRESH/TIME_UNITS
    corner2 = temp.copy()
    corner2[:,0] = corner2[:,0] - DLAT_THRESH
    corner2[:,1] = corner2[:,1] + DLON_THRESH
    corner2[:,2] = corner2[:,2] - DT_THRESH/TIME_UNITS
    corner3 = temp.copy()
    corner3[:,0] = corner3[:,0] + DLAT_THRESH
    corner3[:,1] = corner3[:,1] + DLON_THRESH
    corner3[:,2] = corner3[:,2] - DT_THRESH/TIME_UNITS
    corner4 = temp.copy()
    corner4[:,0] = corner4[:,0] + DLAT_THRESH
    corner4[:,1] = corner4[:,1] - DLON_THRESH
    corner4[:,2] = corner4[:,2] - DT_THRESH/TIME_UNITS
    corner5 = temp.copy()
    corner5[:,0] = corner5[:,0] - DLAT_THRESH
    corner5[:,1] = corner5[:,1] - DLON_THRESH
    corner5[:,2] = corner5[:,2] + DT_THRESH/TIME_UNITS
    corner6 = temp.copy()
    corner6[:,0] = corner6[:,0] - DLAT_THRESH
    corner6[:,1] = corner6[:,1] + DLON_THRESH
    corner6[:,2] = corner6[:,2] + DT_THRESH/TIME_UNITS
    corner7 = temp.copy()
    corner7[:,0] = corner7[:,0] + DLAT_THRESH
    corner7[:,1] = corner7[:,1] + DLON_THRESH
    corner7[:,2] = corner7[:,2] + DT_THRESH/TIME_UNITS
    corner8 = temp.copy()
    corner8[:,0] = corner8[:,0] + DLAT_THRESH
    corner8[:,1] = corner8[:,1] - DLON_THRESH
    corner8[:,2] = corner8[:,2] + DT_THRESH/TIME_UNITS
    
    # For each data point:
    N_cosmic = np.zeros(flight.shape[0])
    T_cosmic = np.nan*np.ones(flight.shape[0])
    dTdz_cosmic = np.nan*np.ones(flight.shape[0])
    for i in np.arange(flight.shape[0]):
        
        # Find the profiles in that point's box, and set the cosmic list index so that it
        # lines up with the row number in the temperature array.
        box = np.stack((corner1[i,:],corner2[i,:],corner3[i,:],corner4[i,:],
                        corner5[i,:],corner6[i,:],corner7[i,:],corner8[i,:]))
        temp = np.array(cosmic_to_load[['Lat','Lon','time_float']])
        inside = Delaunay(box).find_simplex(temp) >= 0
        cosmic_onepoint = cosmic_to_load.reset_index(drop = True)
        cosmic_onepoint['inside'] = inside
        cosmic_onepoint = cosmic_onepoint[cosmic_onepoint.inside == True].drop(['inside'], axis=1)
        indices = np.array(cosmic_onepoint.index)
        N_cosmic[i] = np.size(indices)
        
        # Check that there's at least one profile
        if N_cosmic[i] > 0:
            # Average the profiles together (COSMIC altitude is in kilometers)
            T_profile = np.nanmean(temperatures[indices,:],axis=0)
            dTdz_profile = (T_profile[1:] - T_profile[:-1])/(1000*(altitude[1:] - altitude[:-1]))
            
            # Interpolate to the balloon altitude (balloon altitude is in meters)
            temp_altitude = altitude[~np.isnan(T_profile)]
            if temp_altitude.shape[0] > 1: # make sure there's enough data to do the interpolation
                # check that the data span the balloon altitude:
                if np.nanmax(temp_altitude) >= flight.altitude.iloc[i]/1000 and np.nanmin(temp_altitude) <= flight.altitude.iloc[i]/1000:
                    T_profile = T_profile[~np.isnan(T_profile)]
                    T_cosmic[i] = interpolate.pchip_interpolate(temp_altitude, T_profile, flight.altitude.iloc[i]/1000)
                    
                    temp_altitude = 0.5*(altitude[1:] + altitude[:-1])
                    temp_altitude = temp_altitude[~np.isnan(dTdz_profile)]
                    dTdz_profile = dTdz_profile[~np.isnan(dTdz_profile)]
                    dTdz_cosmic[i] = interpolate.pchip_interpolate(temp_altitude, dTdz_profile,
                                                                   flight.altitude.iloc[i]/1000)
                else: # if the balloon altitude is outside the range
                    N_cosmic[i] = 0
            else: # if there's not enough data
                N_cosmic[i] = 0
        
    backgroundT = flight[['T_cosmic','dTdz_cosmic','N_cosmic']].copy()
    backgroundT['T_cosmic'] = T_cosmic
    backgroundT['dTdz_cosmic'] = dTdz_cosmic
    backgroundT['N_cosmic'] = N_cosmic
    
    return backgroundT # output is T, dTdz, N_COSMIC

### Do the analysis

In [10]:
# Select all flights
segment_ids = flights.segment_id.unique()

# The altitude array for all the profiles has the 0.1km resolution for altitudes
# from 0 to 39.9km:
altitude = np.linspace(0, 39.9, 400)
N_alt = altitude.size

nsegments = segment_ids.shape[0]
for i in np.arange(nsegments):
    print('Fraction complete:', i+1, '/', nsegments, 'segments')
    print('Current time:', datetime.datetime.now())
    print('Segment id:', segment_ids[i])
    flight = flights[flights.segment_id == segment_ids[i]].copy()
    print('Flight start, length:', flight.time.min(), ',', flight.time.max() - flight.time.min())
    print('Flight N:', flight.shape[0])
    
    # Find the list of profiles to load for the flight
    start_time = time()
    cosmic_to_load = flight_find_cosmic(flight, cosmic_list)
    print('Find profiles elapsed time (s): ' + str(time()-start_time))
    print('Number of COSMIC profiles: ', cosmic_to_load.shape[0])

    # Load in those profiles and interpolate them onto the evenly spaced altitude array
    start_time = time()
    temperatures = np.nan*np.ones(shape=(cosmic_to_load.shape[0],N_alt))
    bad_frac = 0
    for j in np.arange(cosmic_to_load.shape[0]):
        if xr.open_dataset(cosmic_to_load.Name.iloc[j]).bad == '1': #check for bad data
            bad_frac = bad_frac + 1/cosmic_to_load.shape[0]
        else:
            temp_T = np.flip(xr.open_dataset(cosmic_to_load.Name.iloc[j]).Temp)
            temp_alt = np.flip(xr.open_dataset(cosmic_to_load.Name.iloc[j]).MSL_alt)
            # Check to make sure there are no repeated altitudes
            temp_T = np.array(temp_T)
            temp_alt = np.array(temp_alt)
            temp_delta_alt = temp_alt[1:] - temp_alt[:-1]
            ind = 1 + np.arange(np.size(temp_alt)-1)
            ind = ind[temp_delta_alt <= 0]
            if np.size(ind) > 0:
                temp_alt = np.delete(temp_alt,ind)
                temp_T = np.delete(temp_T,ind)
            # Delete any NaNs
            ind = np.arange(np.size(temp_alt))
            ind = ind[np.isnan(temp_T) == 1]
            if np.size(ind) > 0:
                temp_alt = np.delete(temp_alt,ind)
                temp_T = np.delete(temp_T,ind)
            temperatures[j,:] = interpolate.pchip_interpolate(temp_alt, temp_T, altitude)
    print('Load profiles elapsed time (s): ' + str(time()-start_time))
    print('Fraction of profiles that have bad data:', bad_frac)

    # Identify the profiles in range for each data point, average those profiles together 
    # and interpolate to the balloon altitudes.
    start_time = time()
    backgroundT = calc_backgroundT(flight, cosmic_to_load, temperatures, altitude)
    flight['T_cosmic'] = backgroundT.T_cosmic
    flight['dTdz_cosmic'] = backgroundT.dTdz_cosmic
    flight['N_cosmic'] = backgroundT.N_cosmic
    flights[flights.segment_id == segment_ids[i]] = flight
    print('Data point analysis elapsed time (s): ' + str(time()-start_time))
    print('Segment mean N_cosmic:', backgroundT.N_cosmic.mean())
    print('')
    
temp = flights[flights.segment_id.isin(segment_ids)].reset_index(drop=True)
temp.to_feather('temp_data/3b_segments_cosmic_2021_6hrSegs.feather')

Fraction complete: 1 / 1560 segments
Current time: 2022-03-18 08:40:51.805857
Segment id: 5135
Flight start, length: 2021-01-14 13:28:35 , 0 days 07:19:15
Flight N: 568
Find profiles elapsed time (s): 0.6853318214416504
Number of COSMIC profiles:  300
Load profiles elapsed time (s): 5.2719480991363525
Fraction of profiles that have bad data: 0.05
Data point analysis elapsed time (s): 1.460486888885498
Segment mean N_cosmic: 253.6056338028169

Fraction complete: 2 / 1560 segments
Current time: 2022-03-18 08:40:59.287542
Segment id: 5137
Flight start, length: 2021-01-15 07:24:09 , 0 days 06:02:53
Flight N: 463
Find profiles elapsed time (s): 0.5385160446166992
Number of COSMIC profiles:  300
Load profiles elapsed time (s): 5.428842067718506
Fraction of profiles that have bad data: 0.043333333333333335
Data point analysis elapsed time (s): 1.2354040145874023
Segment mean N_cosmic: 262.65442764578836

Fraction complete: 3 / 1560 segments
Current time: 2022-03-18 08:41:06.508810
Segment id:

Data point analysis elapsed time (s): 1.3218109607696533
Segment mean N_cosmic: 203.8625

Fraction complete: 19 / 1560 segments
Current time: 2022-03-18 08:43:47.320493
Segment id: 20715
Flight start, length: 2021-01-12 12:26:45 , 0 days 07:37:20
Flight N: 586
Find profiles elapsed time (s): 0.5735390186309814
Number of COSMIC profiles:  279
Load profiles elapsed time (s): 5.764194011688232
Fraction of profiles that have bad data: 0.035842293906810034
Data point analysis elapsed time (s): 1.901686668395996
Segment mean N_cosmic: 225.72013651877134

Fraction complete: 20 / 1560 segments
Current time: 2022-03-18 08:43:55.563091
Segment id: 20724
Flight start, length: 2021-01-15 13:02:38 , 0 days 08:17:29
Flight N: 638
Find profiles elapsed time (s): 0.6252450942993164
Number of COSMIC profiles:  267
Load profiles elapsed time (s): 5.78918719291687
Fraction of profiles that have bad data: 0.03745318352059925
Data point analysis elapsed time (s): 1.7698211669921875
Segment mean N_cosmic: 2

Load profiles elapsed time (s): 6.949436187744141
Fraction of profiles that have bad data: 0.027777777777777776
Data point analysis elapsed time (s): 3.0196378231048584
Segment mean N_cosmic: 178.17060367454067

Fraction complete: 37 / 1560 segments
Current time: 2022-03-18 08:49:17.627843
Segment id: 24091
Flight start, length: 2021-01-24 10:34:12 , 0 days 14:17:49
Flight N: 1113
Find profiles elapsed time (s): 0.5456397533416748
Number of COSMIC profiles:  225
Load profiles elapsed time (s): 4.500372886657715
Fraction of profiles that have bad data: 0.022222222222222223
Data point analysis elapsed time (s): 2.873769998550415
Segment mean N_cosmic: 119.65408805031447

Fraction complete: 38 / 1560 segments
Current time: 2022-03-18 08:49:25.550215
Segment id: 24101
Flight start, length: 2021-01-26 23:40:04 , 0 days 20:36:02
Flight N: 1578
Find profiles elapsed time (s): 0.5427970886230469
Number of COSMIC profiles:  162
Load profiles elapsed time (s): 3.3649330139160156
Fraction of prof

Load profiles elapsed time (s): 0.608410120010376
Fraction of profiles that have bad data: 0.030303030303030304
Data point analysis elapsed time (s): 1.2938721179962158
Segment mean N_cosmic: 16.74387947269303

Fraction complete: 55 / 1560 segments
Current time: 2022-03-18 08:50:50.469735
Segment id: 24186
Flight start, length: 2021-02-19 01:39:51 , 0 days 09:43:46
Flight N: 751
Find profiles elapsed time (s): 0.529170036315918
Number of COSMIC profiles:  6
Load profiles elapsed time (s): 0.13615202903747559
Fraction of profiles that have bad data: 0
Data point analysis elapsed time (s): 1.4695518016815186
Segment mean N_cosmic: 0.2436750998668442

Fraction complete: 56 / 1560 segments
Current time: 2022-03-18 08:50:52.607023
Segment id: 24190
Flight start, length: 2021-02-20 00:17:49 , 0 days 06:45:46
Flight N: 522
Find profiles elapsed time (s): 0.5202541351318359
Number of COSMIC profiles:  0
Load profiles elapsed time (s): 6.413459777832031e-05
Fraction of profiles that have bad da

Find profiles elapsed time (s): 0.5373649597167969
Number of COSMIC profiles:  234
Load profiles elapsed time (s): 4.699145793914795
Fraction of profiles that have bad data: 0.008547008547008548
Data point analysis elapsed time (s): 2.1856942176818848
Segment mean N_cosmic: 169.32655502392345

Fraction complete: 73 / 1560 segments
Current time: 2022-03-18 08:53:11.350013
Segment id: 24903
Flight start, length: 2021-02-22 01:42:51 , 0 days 06:28:48
Flight N: 510
Find profiles elapsed time (s): 0.5468201637268066
Number of COSMIC profiles:  237
Load profiles elapsed time (s): 5.131812810897827
Fraction of profiles that have bad data: 0.012658227848101264
Data point analysis elapsed time (s): 1.4263250827789307
Segment mean N_cosmic: 170.24117647058824

Fraction complete: 74 / 1560 segments
Current time: 2022-03-18 08:53:18.457438
Segment id: 24922
Flight start, length: 2021-02-28 10:53:11 , 0 days 07:40:43
Flight N: 594
Find profiles elapsed time (s): 0.542341947555542
Number of COSMIC p

...

Find profiles elapsed time (s): 0.5309591293334961
Number of COSMIC profiles:  281
Load profiles elapsed time (s): 4.652696132659912
Fraction of profiles that have bad data: 0.04626334519572954
Data point analysis elapsed time (s): 1.8066470623016357
Segment mean N_cosmic: 215.15528781793842

Fraction complete: 1477 / 1560 segments
Current time: 2022-03-18 12:39:02.315123
Segment id: 60502
Flight start, length: 2021-04-06 20:22:21 , 0 days 11:09:50
Flight N: 866
Find profiles elapsed time (s): 0.5363907814025879
Number of COSMIC profiles:  349
Load profiles elapsed time (s): 5.218969106674194
Fraction of profiles that have bad data: 0.03151862464183382
Data point analysis elapsed time (s): 2.08008074760437
Segment mean N_cosmic: 234.3418013856813

Fraction complete: 1478 / 1560 segments
Current time: 2022-03-18 12:39:10.153109
Segment id: 60503
Flight start, length: 2021-04-07 09:33:48 , 0 days 18:20:51
Flight N: 1399
Find profiles elapsed time (s): 0.5358788967132568
Number of COSMIC 

Find profiles elapsed time (s): 0.5310368537902832
Number of COSMIC profiles:  377
Load profiles elapsed time (s): 6.965196132659912
Fraction of profiles that have bad data: 0.02652519893899204
Data point analysis elapsed time (s): 2.964042901992798
Segment mean N_cosmic: 194.84700315457414

Fraction complete: 1495 / 1560 segments
Current time: 2022-03-18 12:41:31.020874
Segment id: 60549
Flight start, length: 2021-04-22 09:40:19 , 0 days 07:45:54
Flight N: 585
Find profiles elapsed time (s): 0.5320656299591064
Number of COSMIC profiles:  319
Load profiles elapsed time (s): 5.794526815414429
Fraction of profiles that have bad data: 0.01567398119122257
Data point analysis elapsed time (s): 1.467127799987793
Segment mean N_cosmic: 248.0102564102564

Fraction complete: 1496 / 1560 segments
Current time: 2022-03-18 12:41:38.817205
Segment id: 60550
Flight start, length: 2021-04-22 17:39:37 , 0 days 06:21:45
Flight N: 490
Find profiles elapsed time (s): 0.5367159843444824
Number of COSMIC p

Find profiles elapsed time (s): 0.5545527935028076
Number of COSMIC profiles:  432
Load profiles elapsed time (s): 7.1225128173828125
Fraction of profiles that have bad data: 0.020833333333333332
Data point analysis elapsed time (s): 1.6529803276062012
Segment mean N_cosmic: 243.9970238095238

Fraction complete: 1513 / 1560 segments
Current time: 2022-03-18 12:43:43.699362
Segment id: 60610
Flight start, length: 2021-05-07 12:41:55 , 0 days 07:52:31
Flight N: 604
Find profiles elapsed time (s): 0.5489373207092285
Number of COSMIC profiles:  414
Load profiles elapsed time (s): 6.873522043228149
Fraction of profiles that have bad data: 0.021739130434782608
Data point analysis elapsed time (s): 1.4840588569641113
Segment mean N_cosmic: 227.85927152317882

Fraction complete: 1514 / 1560 segments
Current time: 2022-03-18 12:43:52.608324
Segment id: 60627
Flight start, length: 2021-05-11 22:10:11 , 0 days 11:10:57
Flight N: 878
Find profiles elapsed time (s): 0.5271320343017578
Number of COS

Find profiles elapsed time (s): 0.5597870349884033
Number of COSMIC profiles:  437
Load profiles elapsed time (s): 7.856580972671509
Fraction of profiles that have bad data: 0.07551487414187646
Data point analysis elapsed time (s): 4.273177862167358
Segment mean N_cosmic: 256.32

Fraction complete: 1531 / 1560 segments
Current time: 2022-03-18 12:47:35.931001
Segment id: 60767
Flight start, length: 2021-02-07 11:20:10 , 0 days 10:23:51
Flight N: 813
Find profiles elapsed time (s): 0.5517871379852295
Number of COSMIC profiles:  285
Load profiles elapsed time (s): 5.171539783477783
Fraction of profiles that have bad data: 0.08421052631578943
Data point analysis elapsed time (s): 1.9177279472351074
Segment mean N_cosmic: 177.51660516605165

Fraction complete: 1532 / 1560 segments
Current time: 2022-03-18 12:47:43.574338
Segment id: 60771
Flight start, length: 2021-02-08 11:18:40 , 0 days 07:46:24
Flight N: 604
Find profiles elapsed time (s): 0.5574221611022949
Number of COSMIC profiles:  

Find profiles elapsed time (s): 0.5817248821258545
Number of COSMIC profiles:  449
Load profiles elapsed time (s): 8.271373987197876
Fraction of profiles that have bad data: 0.055679287305122505
Data point analysis elapsed time (s): 1.8976149559020996
Segment mean N_cosmic: 298.3212598425197

Fraction complete: 1549 / 1560 segments
Current time: 2022-03-18 12:50:31.349093
Segment id: 60830
Flight start, length: 2021-02-25 23:26:37 , 0 days 12:22:53
Flight N: 958
Find profiles elapsed time (s): 0.6079919338226318
Number of COSMIC profiles:  490
Load profiles elapsed time (s): 9.728821992874146
Fraction of profiles that have bad data: 0.057142857142857106
Data point analysis elapsed time (s): 2.6543171405792236
Segment mean N_cosmic: 243.68371607515658

Fraction complete: 1550 / 1560 segments
Current time: 2022-03-18 12:50:44.343016
Segment id: 60831
Flight start, length: 2021-02-26 13:20:05 , 0 days 08:59:02
Flight N: 695
Find profiles elapsed time (s): 0.6222250461578369
Number of COSM

### Load in each of the processed years and combine them into one dataframe

In [14]:
temp1 = pd.read_feather('temp_data/3b_segments_cosmic_2011-2018.feather')
temp2 = pd.read_feather('temp_data/3b_segments_cosmic_2019.feather')
temp3 = pd.read_feather('temp_data/3b_segments_cosmic_2020_Jan-Jun.feather')
temp4 = pd.read_feather('temp_data/3b_segments_cosmic_2020_Jul-Dec_6hrSegs.feather')
temp5 = pd.read_feather('temp_data/3b_segments_cosmic_2021_6hrSegs.feather')

# Increment the segment IDs from one file to another, so they are still unique
temp2.segment_id = temp2.segment_id + temp1.segment_id.max() + 1
temp3.segment_id = temp3.segment_id + temp2.segment_id.max() + 1
temp4.segment_id = temp4.segment_id + temp3.segment_id.max() + 1
temp5.segment_id = temp5.segment_id + temp4.segment_id.max() + 1

frames = [temp1, temp2, temp3, temp4, temp5]
flights = pd.concat(frames, ignore_index = True)
flights = flights.drop(columns='time_float')

### Interpolate temperatures to fill in NaNs (where there is no COSMIC data).
N_cosmic will remain zero for these data points.

In [15]:
def interp_cosmic(segment):
    
    # Delete rows if the timestamp is repeated
    temp = segment.time.diff()
    temp = temp[temp == pd.Timedelta('0 days 00:00:00')]
    segment = segment.drop(labels = temp.index)
    
    # Select the values of T and dTdz that aren't NaN (N_cosmic > 0)
    time_nonans = segment[segment.N_cosmic>0].time
    T = segment[segment.N_cosmic>0].T_cosmic
    dTdz = segment[segment.N_cosmic>0].dTdz_cosmic
    
    # If there are no COSMIC profiles for the flight, or only one data point with COSMIC profiles,
    # use unrealistic values for T and dTdz.
    if time_nonans.shape[0] < 2:
        T_interp = T_COSMIC_DUMMY
        dTdz_interp = DTDZ_COSMIC_DUMMY
    
    # Otherwise, do the interpolation
    else:
        # Times must be converted to a float before being interpolated, then back to datetime64 after
        time_float = np.array(segment.time).astype('float')
        time_nonans_float = np.array(time_nonans).astype('float')
        time_float = time_float - time_nonans_float[0]
        time_nonans_float = time_nonans_float - time_nonans_float[0]
        
        # Do the interpolation
        T_interp = interpolate.pchip_interpolate(time_nonans_float, T, time_float)
        dTdz_interp = interpolate.pchip_interpolate(time_nonans_float, dTdz, time_float)
    
    # Update the table
    segment['T_cosmic'] = T_interp
    segment['dTdz_cosmic'] = dTdz_interp
    
    return segment

flights = flights.groupby('segment_id').progress_apply(interp_cosmic).reset_index(drop=True)

  0%|          | 0/124073 [00:00<?, ?it/s]

### Save the data

In [16]:
flights.to_feather('temp_data/3b_segments_cosmic.feather')
flights